In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
from collections import deque
import csv
import os
from copy import deepcopy



# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technical but must be specific so the customer understands the project)

In [3]:
#Create a neural network that plays tetris as long as possible

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [1]:
# data = pd.read_csv("src/data_0.csv")
# data.head()

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [2]:
# plt.figure(figsize=(10,6))
# plt.plot(data["avg_fit"], label="Average Fitness", marker="o")
# plt.plot(data["top_fit"], label="Top Fitness", marker="s")
# plt.plot(data["elite_fit"], label="Elite Fitness", linestyle="--")
# plt.title("Tetris Reinforcement Learning Training Progress")
# plt.xlabel("Generation")
# plt.ylabel("Fitness Score")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# #Highest fitness is around the 23rd generation

# 4.Prepare the Data


Apply any data transformations and explain what and why


In [11]:
# Using reinforcement learning so don't need parameters just the board
def bool_to_np(board):
    return np.array(board, dtype=np.float32).flatten()

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


In [7]:
class RL_AI(nn.Module):
    def __init__(self, input_size=240, hidden_sizes=[128, 256, 256, 128], output_size=1, lr=0.001):
        super(RL_AI, self).__init__()
        layers = []
        last_size = input_size
        for h in hidden_sizes:
            layers.append(nn.Linear(last_size, h))
            layers.append(nn.ReLU(inplace=True))
            last_size = h
        layers.append(nn.Linear(last_size, output_size))
        self.model = nn.Sequential(*layers)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()
        self._create_weights()

    def _create_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        if x.ndim == 1:
            x = x.unsqueeze(0)
        return self.model(x)

    def valuate(self, board_np):
        board_tensor = torch.tensor(board_np, dtype=torch.float32).unsqueeze(0)
        return self.forward(board_tensor).item()

    def get_best_move(self, board, piece):
        best_x = -1
        max_value = -float('inf')
        best_piece = None

        for i in range(4):
            piece = piece.get_next_rotation()
            for x in range(board.width):
                try:
                    y = board.drop_height(piece, x)
                except:
                    continue

                board_copy = deepcopy(board.board)
                for pos in piece.body:
                    board_copy[y + pos[1]][x + pos[0]] = True

                np_board = bool_to_np(board_copy)
                value = self.valuate(np_board)

                if value > max_value:
                    max_value = value
                    best_x = x
                    best_piece = piece

        return best_x, best_piece

    def train_step(self, state, target_value):
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        target_tensor = torch.tensor([[target_value]], dtype=torch.float32)
        pred = self.forward(state_tensor)
        loss = self.loss_fn(pred, target_tensor)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

#The model doesn't learn properly yet, however it simulates training

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best results possible 


In [8]:
class RL_AI(nn.Module):
    def __init__(self, input_size=240, hidden_sizes=[128, 256, 256, 128],
                 output_size=1, lr=0.001, gamma=0.99, epsilon=0.2):
        super(RL_AI, self).__init__()
        self.gamma = gamma
        self.epsilon = epsilon 

        layers = []
        last = input_size
        for h in hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU(inplace=True))
            last = h
        layers.append(nn.Linear(last, output_size))
        self.model = nn.Sequential(*layers)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        if x.ndim == 1:
            x = x.unsqueeze(0)
        return self.model(x)

    def valuate(self, board_np):
        return self.forward(board_np).item()

    def get_best_move(self, board, piece):
        best_x = -1
        best_piece = None
        best_value = -float('inf')
        moves = []

        for r in range(4):
            rotated_piece = piece if r == 0 else piece.get_next_rotation()
            for x in range(board.width):
                try:
                    y = board.drop_height(rotated_piece, x)
                except:
                    continue
                temp_board = deepcopy(board)
                result = temp_board.place(x, y, rotated_piece)
                if isinstance(result, Exception):
                    continue
                cleared = temp_board.clear_rows()
                np_board = bool_to_np(temp_board.board)
                reward = cleared * 100.0 - self.board_penalty(temp_board)
                moves.append((x, rotated_piece, np_board, reward))

        if random.random() < self.epsilon:
            return random.choice(moves)[:2]

        for x, p, np_board, reward in moves:
            value = self.forward(np_board).item() + reward
            if value > best_value:
                best_value, best_x, best_piece = value, x, p

        return best_x, best_piece

    def board_penalty(self, board):
        heights = board.heights
        holes = self.count_holes(board.board)
        bumpiness = sum(abs(heights[i] - heights[i+1]) for i in range(board.width-1))
        return 0.4 * max(heights) + 0.7 * holes + 0.3 * bumpiness

    def count_holes(self, board_array):
        holes = 0
        for col in range(len(board_array[0])):
            block_found = False
            for row in range(len(board_array)):
                if board_array[row][col]:
                    block_found = True
                elif block_found:
                    holes += 1
        return holes

    def train_step(self, state, reward, next_state=None, done=False):
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        reward_tensor = torch.tensor([[reward]], dtype=torch.float32)
        q_val = self.forward(state_tensor)

        if next_state is not None:
            next_state_tensor = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
            with torch.no_grad():
                q_next = self.forward(next_state_tensor)
                q_target = reward_tensor + (0 if done else self.gamma * q_next)
        else:
            q_target = reward_tensor

        loss = self.loss_fn(q_val, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

#Tried penalizing more and increasing the epsilon rate but it is still not learning properly due to lack of replay function

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def bool_to_np_trimmed(board):
    trimmed_board = board[:20] 
    return np.array(trimmed_board, dtype=np.float32).flatten()

class DQN_AI(nn.Module):
    def __init__(self, state_dim=200, action_dim=10, hidden_sizes=[128, 128], lr=0.001, gamma=0.99, epsilon=0.1):
        super(DQN_AI, self).__init__()
        self.gamma = gamma
        self.epsilon = epsilon
        self.action_dim = action_dim

        layers = []
        last = state_dim
        for h in hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, action_dim))
        self.model = nn.Sequential(*layers).to(device)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()
        self.memory = []
        self.batch_size = 64
        self.max_memory = 10000

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32, device=device)
        if x.ndim == 1:
            x = x.unsqueeze(0)
        return self.model(x)

    def act(self, state):
        state_tensor = torch.tensor(state, dtype=torch.float32, device=device)
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        with torch.no_grad():
            q_values = self.forward(state_tensor)
            return int(torch.argmax(q_values, dim=1).item())

    def remember(self, state, action, reward, next_state, done):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(states, dtype=torch.float32, device=device)
        actions = torch.tensor(actions, dtype=torch.long, device=device).unsqueeze(1)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device).unsqueeze(1)
        next_states = torch.tensor(next_states, dtype=torch.float32, device=device)
        dones = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

        q_vals = self.model(states).gather(1, actions)
        with torch.no_grad():
            q_next = self.model(next_states).max(1)[0].unsqueeze(1)
            q_target = rewards + (1 - dones) * self.gamma * q_next

        loss = self.loss_fn(q_vals, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

#Found out that reinforcement learning needs to be combined with QN learning to learn properly, currently still doesn't learn

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def bool_to_np_trimmed(board):

    trimmed_board = board[:20] 
    return np.array(trimmed_board, dtype=np.float32).flatten()

def compute_board_features(board_obj):
    flat_board = bool_to_np_trimmed(board_obj.board)
    heights = board_obj.heights
    holes = 0
    for col in range(board_obj.width):
        block_found = False
        for row in range(board_obj.height):
            if board_obj.board[row][col]:
                block_found = True
            elif block_found:
                holes += 1
    bumpiness = sum(abs(heights[i] - heights[i + 1]) for i in range(len(heights) - 1))
    return np.concatenate([flat_board, np.array(heights, dtype=np.float32), np.array([holes, bumpiness], dtype=np.float32)])


class DQN_AI(nn.Module):
    def __init__(self, state_dim=212, action_dim=10, hidden_sizes=[128, 128], lr=0.001, gamma=0.99, epsilon_start=1.0, epsilon_end=0.1):
        super(DQN_AI, self).__init__()
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.action_dim = action_dim

        # Q-network
        layers = []
        last = state_dim
        for h in hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, action_dim))
        self.model = nn.Sequential(*layers).to(device)

        # Target network
        self.target_model = deepcopy(self.model)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()
        self.memory = []
        self.batch_size = 128
        self.max_memory = 10000

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32, device=device)
        if x.ndim == 1:
            x = x.unsqueeze(0)
        return self.model(x)

    def act(self, state, training=True):
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        with torch.no_grad():
            q_values = self.forward(state)
            return int(torch.argmax(q_values, dim=1).item())

    def remember(self, state, action, reward, next_state, done):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(states, dtype=torch.float32, device=device)
        actions = torch.tensor(actions, dtype=torch.long, device=device).unsqueeze(1)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device).unsqueeze(1)
        next_states = torch.tensor(next_states, dtype=torch.float32, device=device)
        dones = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

        q_vals = self.model(states).gather(1, actions)
        with torch.no_grad():
            q_next = self.target_model(next_states).max(1)[0].unsqueeze(1)
            q_target = rewards + (1 - dones) * self.gamma * q_next

        loss = self.loss_fn(q_vals, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def decay_epsilon(self, episode, max_episodes):
        self.epsilon = max(self.epsilon_end, self.epsilon_start * (1 - episode / max_episodes))

#Implemented eplison decay and the model properly learns now

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def bool_to_np_trimmed(board):
    trimmed_board = board[:20]  # keep top 20 rows
    return np.array(trimmed_board, dtype=np.float32).flatten()

def compute_board_features(board_obj):
    flat_board = bool_to_np_trimmed(board_obj.board)
    heights = board_obj.heights
    holes = 0
    for col in range(board_obj.width):
        block_found = False
        for row in range(board_obj.height):
            if board_obj.board[row][col]:
                block_found = True
            elif block_found:
                holes += 1
    bumpiness = sum(abs(heights[i] - heights[i + 1]) for i in range(len(heights) - 1))
    return np.concatenate([flat_board, np.array(heights, dtype=np.float32), np.array([holes, bumpiness], dtype=np.float32)])

class DQN_AI(nn.Module):
    def __init__(self, state_dim=212, action_dim=10, hidden_sizes=[128, 128], lr=0.001, gamma=0.99, epsilon_start=1.0, epsilon_end=0.1):
        super(DQN_AI, self).__init__()
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.action_dim = action_dim

        # Q-network
        layers = []
        last = state_dim
        for h in hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, action_dim))
        self.model = nn.Sequential(*layers).to(device)

        # Target network
        self.target_model = deepcopy(self.model)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()
        self.memory = []
        self.batch_size = 128
        self.max_memory = 10000

    def forward(self, x):
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32, device=device)
        if x.ndim == 1:
            x = x.unsqueeze(0)
        return self.model(x)

    def act(self, state, training=True):
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        with torch.no_grad():
            q_values = self.forward(state)
            return int(torch.argmax(q_values, dim=1).item())

    def get_best_move(self, board_obj, piece):
        best_value = -float('inf')
        best_x = 0
        for x in range(board_obj.width):
            try:
                y = board_obj.drop_height(piece, x)
                temp_board = board_obj.clone()  
                temp_board.place(x, y, piece)
                features = compute_board_features(temp_board)
                value = self.forward(features).item()
                if value > best_value:
                    best_value = value
                    best_x = x
            except:
                continue 
        return best_x, piece

    def remember(self, state, action, reward, next_state, done):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.tensor(states, dtype=torch.float32, device=device)
        actions = torch.tensor(actions, dtype=torch.long, device=device).unsqueeze(1)
        rewards = torch.tensor(rewards, dtype=torch.float32, device=device).unsqueeze(1)
        next_states = torch.tensor(next_states, dtype=torch.float32, device=device)
        dones = torch.tensor(dones, dtype=torch.float32, device=device).unsqueeze(1)

        q_vals = self.model(states).gather(1, actions)
        with torch.no_grad():
            q_next = self.target_model(next_states).max(1)[0].unsqueeze(1)
            q_target = rewards + (1 - dones) * self.gamma * q_next

        loss = self.loss_fn(q_vals, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def decay_epsilon(self, episode, max_episodes):
        self.epsilon = max(self.epsilon_end, self.epsilon_start * (1 - episode / max_episodes))

# Implemented best move function so the model plays the game properly after training

# 7. Present
In a customer facing Document provide summary of finding and detail approach taken


In [30]:
"""Tasked with making a neural network that plays tetris for as long as possible  
The data was outdated and using reinforcement learning, so do not need to check parameters
Went through 6 iterations:
1: It could simulate gameplay through a simple neural network but could not learn
2: It could evaluate the moves better since the parameters such as epsilon were increased, but could not learn
3: Added q learning so it could remember and learn through the replay function
4: Implemented missing epislon decay function to update the rewards
5: Reimplemented the best move function so the model can be tested
6: Work in progress, currently attempting to train the model with 50000 episodes, currently trained at 5000 and runs for a second
"""

'Tasked with making a neural network that plays tetris for as long as possible  \nThe data was outdated and using reinforcement learning, so do not need to check parameters\nWent through 4 iterations:\n'

# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [29]:
#from game import Game
import torch

def inference(agent=None):
    if agent is None:
        agent = DQN_AI(212, 10)
        agent.load_state_dict(torch.load('5000EpisodeAgent.pth'))
    game = Game(mode="DQN", agent=agent)
    pieces_dropped, rows_cleared = game.run()
    reward = pieces_dropped + rows_cleared
    print(f"Test -> Pieces: {pieces_dropped}, Rows: {rows_cleared}, Reward: {reward}")

if __name__ == "__main__":
    inference()


# Inference code which loads the model, called test.py in actual project (tetris_a/src) and incoroprates the game, board and piece files from the tetris game
# https://lab2.check-ai.com/user/260721/lab/tree/Machine-Learning-AI-Class/tetris_a/src/test.py

NameError: name 'Game' is not defined